In [1]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry


response = requests.get('https://afcot.org/en/member-directory/')
soup = BeautifulSoup(response.text, 'html.parser')

In [2]:
items = soup.find_all("div", class_="element-item")

In [1]:
def safe_get_text(obj, name=None, attrs={}, recursive=True, string=None,
             **kwargs):
    r = None
    l = obj.find_all(name, attrs, recursive, string, 1, **kwargs)
    if l:
        r = l[0].text.replace("\r\n", "  ")
    return r


def safe_get_href(obj, name=None, attrs={}, recursive=True, string=None,
             **kwargs):
    r = None
    l = obj.find_all(name, attrs, recursive, string, 1, **kwargs)
    if l:
        r = l[0]["href"].replace("\r\n", "  ")
    return r


In [2]:
import pandas as pd
target_list = ["membre-name", 
               "membre-cat",
               "membre-adresse",
               "membre-phone",
               "membre-fax",
               "membre-mail",
               "membre-web"]
# 주어진 target_list를 기반으로 빈 DataFrame 생성
df = pd.DataFrame(columns=target_list)

# items 리스트에 대한 반복문을 통해 데이터를 가져와 DataFrame에 추가
for i, item in enumerate(items):
    member_data = {}
    
    member_data["membre-name"] = safe_get_text(item, "p", class_="membre-name")
    member_data["membre-cat"] = safe_get_text(item, "p", class_="membre-cat")
    member_data["membre-adresse"] = safe_get_text(item, "p", class_="membre-adresse")
    member_data["membre-phone"] = safe_get_text(item, "p", class_="membre-phone")
    member_data["membre-fax"] = safe_get_text(item, "p", class_="membre-fax")
    member_data["membre-mail"] = safe_get_href(item, "a", class_="membre-mail")
    member_data["membre-web"] = safe_get_href(item, "a", class_="membre-web")

    # DataFrame에 데이터 추가
    df = df.append(member_data, ignore_index=True)



NameError: name 'items' is not defined

In [64]:
df.to_csv('./afcot.csv')

In [12]:
member_type_list = [t for t in item['class'] if t != "element-item"]
member_type = member_type_list[0] if member_type_list else None


'membres-dirigeants'

In [ ]:

table = soup.find('table', class_='eb-table')
tbody = table.find('tbody')
rows = tbody.find_all('tr')

df = None
for row in rows:
    td = row.find('td', class_='td-subject')
    a = td.find('a')
    row_date = extract_date_from_string(a.text.strip())

    if target_date_str == row_date:
        print('대상 데이터 있음', row_date)
        cols, values_list = crawling_page(a['href'])
        df = pd.DataFrame(values_list, columns=cols)
        break